In [ ]:
from sage.misc.prandom import randint
from sage.rings.all import ZZ, IntegerModRing, RR, PolynomialRing, GF
from sage.arith.all import next_prime, euler_phi, crt, xgcd
from sage.functions.log import log
from sage.functions.other import ceil
from sage.misc.functional import cyclotomic_polynomial, round
import numpy as np
from random import randint

# Parameter Generation

In [ ]:
d = 3
m = 3
N = 2**6
n = euler_phi(N)
log_q = 9
evaluation_degree = d*m
D = d*m

In [ ]:
var('Z')
g = cyclotomic_polynomial(N, var='Z')

In [ ]:
primes = []
q = 1
i = 1
while len(primes) < 1:
    p = N*i+1
    if p.is_prime():
        q *= p
        primes.append(p)
    i += 1
primes

In [ ]:
Z_q = IntegerModRing(q)
Q = Z_q['Z'].quo(g,Z)

In [ ]:
var('Y')
h = Q['Y'](1)
monomials = []
assert D < q
for i in range(D):
    mon = (Q[Y](Y) - Q[Y](i))
    h *= mon
    monomials.append(mon)
R = Q['Y'].quo(h,Y)

# BV11

In [ ]:
s = Q.random_element()

In [ ]:
def encrypt(s,mu):
    assert mu in Z_q
    assert s in Q
    a = R(Q.random_element())
    
    # TODO make the noise real
    e = randint(0,2)

    b = a*R(s)+d*e+mu
    return -a*R(Y)+b

In [ ]:
c1 = encrypt(s,1)
c2 = encrypt(s,2)
c = c1 * c2

In [ ]:
def decrypt(s,c):
    a = Q(c.lift()(Y=s)).lift()(Z=0).lift()
    return mod(a,d)

In [ ]:
decrypt(s,c)

# Database Interpolation

In [ ]:
P = PolynomialRing(GF(d),m,'x')
vars = P.gens()
vars[0]

In [ ]:
a=np.array([[1,5,3],
[2,4,7],
[7,8,1]])
b=[1,5,3]
def multivariate_interpolation(a, vars, m):
    if m==1:
        return GF(d)[vars[0]].lagrange_polynomial([(i,a[i])for i in range(d)])
    else:
        for x in GF(d)**(m-1):
            g_x = GF(d)[vars[-1]].lagrange_polynomial([(i,a[tuple(x)][i])for i in range(d)])
            print(g_x)
multivariate_interpolation(a,vars,2)


In [ ]:
np.array([[1,5,3]])[[0,0]]

In [ ]:
f = P(x[0]*x[1]*2+x[1]*3+x[0]*4+x[2]*x[1]*5)
f

# Eval

In [ ]:
def normal_eval(Q,f,cs):
    f_prime =f.change_ring(Q)
    return f_prime(cs)

In [ ]:
normal_eval(R,f,[c1,c2,c1])

In [ ]:
w_n = Z_q.multiplicative_generator()**((primes[0]-1)//N)
assert w_n.multiplicative_order() == N
frac_qs = [Z_q[Z](Z)-Z_q[Z](w_n**i) for i in IntegerModRing(N).list_of_elements_of_multiplicative_group()]


In [ ]:
def split_Q(c):
    return [Z_q[Z].quo(frac_q,Z)(c) for frac_q in frac_qs]

def combine_Q(cs):
    return Q(crt([c.lift() for c in cs],frac_qs))

def split_eval_Q(f,cs):
    cs_split = [split_Q(c) for c in cs]
    result_split = []
    for i in range(len(cs_split[0])):
        result_split.append(normal_eval(IntegerModRing(q),f,[cs_split[j][i] for j in range(len(cs_split))]))
    return combine_Q(result_split)


In [ ]:
CRT = []
for i in range(len(monomials)):
    a = [1 if i == j else 0 for j in range(len(monomials))]
    CRT.append(Q[Y](Z_q[Y,Z](crt(a,[Z_q[Y](Z_q[Y,Z](mon)) for mon in monomials]))))

In [ ]:
def split_R(c):
    return [Q[Y].quo(mon)(c.lift()) for mon in monomials]

def combine_R(cs):
    result = Q[Y](0)
    for i in range(len(cs)):
        result += cs[i].lift()*CRT[i]
    return result
    

In [ ]:
combine_R(split_R(c))